In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def add_layer(inputs,in_size,out_size,activation_function=None):
    # 初始值随机比全部为零要好
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size,out_size]),name='W')
        with tf.name_scope('biases'):
            # 初始值全部为0.1 同上
            biases = tf.Variable(tf.zeros([1,out_size]) + 0.1,name='b')
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs,Weights),biases)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        return outputs

# 有点儿像函数内的变量
    # 也可以用来选择部分数据训练模型
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32,[None,1],name='x_input')
    ys = tf.placeholder(tf.float32,[None,1],name='y_input')

# 第一个隐含层
l1 = add_layer(xs,1,10,activation_function=tf.nn.relu)
# 输出层
prediction = add_layer(l1,10,1,activation_function=None)

with tf.name_scope('loss'):
    # lose function
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),reduction_indices=[1]))
with tf.name_scope('train'):
    # 优化器参数 学习率，目标是minimize loss
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

init = tf.global_variables_initializer()
sess = tf.Session()

# 重要
writer = tf.summary.FileWriter("E://",sess.graph)

sess.run(init)


In [1]:
# Classification

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# number 1 to 10 data
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)
print("download ok")

def add_layer(inputs,in_size,out_size,activation_function=None):
    # 初始值随机比全部为零要好
    Weights = tf.Variable(tf.random_normal([in_size,out_size]),name='W')
    # 初始值全部为0.1 同上
    biases = tf.Variable(tf.zeros([1,out_size]) + 0.1,name='b')
    Wx_plus_b = tf.add(tf.matmul(inputs,Weights),biases)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pre = sess.run(prediction,feed_dict={xs:v_xs})
    correct_prediction = tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    result = sess.run(accuracy,feed_dict={xs:v_xs,ys:v_ys})
    return result    
    
# define placeholder for inputs to netword
xs = tf.placeholder(tf.float32,[None,784]) # 28x28
ys = tf.placeholder(tf.float32,[None,10])

# add output layer
prediction = add_layer(xs,784,10,activation_function=tf.nn.softmax)

# the error between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),reduction_indices=[1])) # loss
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)



sess = tf.Session()
# important step
sess.run(tf.initialize_all_variables())

for i in range(1000):
    batch_xs,batch_ys = mnist.train.next_batch(100) # 每次学习一百个
    sess.run(train_step,feed_dict={xs:batch_xs,ys:batch_ys})
    if i % 50 == 0:
        print(compute_accuracy(mnist.test.images,mnist.test.labels))



Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
download ok
Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.0896
0.6473
0.7543
0.7852
0.8045
0.8245
0.8362
0.8433
0.85
0.8584
0.8554
0.863
0.8643
0.8679
0.8687
0.8736
0.8727
0.8763
0.8798
0.8767


In [2]:
# Dropout

import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelBinarizer

# load data
digits = load_digits()
X = digits.data
y = digits.target
y = LabelBinarizer().fit_transform(y)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = .3)


def add_layer(inputs,in_size,out_size,activation_function=None):
    # 初始值随机比全部为零要好
    Weights = tf.Variable(tf.random_normal([in_size,out_size]),name='W')
    # 初始值全部为0.1 同上
    biases = tf.Variable(tf.zeros([1,out_size]) + 0.1,name='b')
    Wx_plus_b = tf.add(tf.matmul(inputs,Weights),biases)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    tf.histogram_summary(layer_name + 'outputs',outputs)
    return outputs


# define placeholder for inputs to netword
xs = tf.placeholder(tf.float32,[None,784]) # 28x28
ys = tf.placeholder(tf.float32,[None,10])

# add output layer
l1 = add_layer(xs,64,100,'l1',activation_function=tf.nn.tanh)
prediction = add_layer(l1,100,10,'l2',activation_function=tf.nn.softmax)

# the error between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),reduction_indices=[1])) # loss
tf.scalar_summary('loss',cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.6).minimize(cross_entropy)



sess = tf.Session()
merged = tf.merge_all_summaries()

sess.run(tf.initialize_all_variables())

for i in range(1000):
    # batch_xs,batch_ys = mnist.train.next_batch(100) # 每次学习一百个
    sess.run(train_step,feed_dict={xs:batch_xs,ys:batch_ys})
    if i % 50 == 0:
        train_result = sess.run(merged,feed_dict={xs:X_train,ys:y_train})
        test_result = sess.run(merged,feed_dict={xs:X_test,ys:y_test})



ImportError: No module named 'sklearn'